# Self Study 5


In this self study we implement and test a simple Markov network model for node prediction, and a Gibbs sampling inference (prediction) process. 

For this material there is no direct support from scikit learn, so we have to do a little more coding ourselves than before ...

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from networkx import read_edgelist
import random

As in the previous self study we use the Lazega lawyer network. This time we load it in a slightly different format, in which we first just construct an networkx graph from the 'friendship' edges, and constuct a separate Pandas dataframe for the node attributes. The process of how attributes from this table are turned into attributes for the nodes in the networkx graph can serve as a template for how to construct additional node attributes (not necessarily only the predefined ones from lazega-attributes.txt). 


In [ ]:
lazega=read_edgelist('lazega-friends.edges',nodetype=int) 
node_atts=pd.read_csv("lazega-attributes.txt", sep=' ')
display(node_atts)

Now we annotate the nodes in the graph with some selected attributes (later on, if you want to do more, you can consider additional attributes). 

In [ ]:

for i in range(node_atts.shape[0]):
    lazega.add_node(node_atts.loc[i,'nodeID'], gender=node_atts.loc[i,'nodeGender'])
    lazega.add_node(node_atts.loc[i,'nodeID'], office=node_atts.loc[i,'nodeOffice'])
    lazega.add_node(node_atts.loc[i,'nodeID'], true_practice=node_atts.loc[i,'nodePractice'])

The 'practice' attribute is going to be our class label that we want to predict. Therfore, we pretend that this attribute is not observed for some randomly selected nodes. We do this by adding a new attribute 'observed_practice' which has either the true practice value, or 'nan' for unobserved.

In [ ]:
random.seed(5)
for i in range(node_atts.shape[0]):
    if random.random() > 0.4:
        lazega.add_node(node_atts.loc[i,'nodeID'], observed_practice=node_atts.loc[i,'nodePractice'])
    else:
        lazega.add_node(node_atts.loc[i,'nodeID'], observed_practice=np.nan)

We now want to predict the missing 'practice' labels using Gibbs sampling on a Markov network model. We will first need to initialize values for the unobserved attributes. 

**Task 1:** complete the following function:

In [ ]:
def init_practice():
    # This function should add an additional attribute predicted_practice.
    # The value of predicted_practices is the same as observed_practice when observed_practice != nan. Otherwise, 
    # predicted_practice is a randomly sampled 'practice' value.

We next define some potential functions. Mostly we will be using node or edge potentials, i.e., potential functions that
depend only on the attributes of a single node, or on the attributes of two nodes (connected by an edge). 

Instead of defining the potential function directly, it is more convenient to first define the log of the potential function. Then, instead of taking a big product (cf. slide 7), one can first take the sum of relevant potentials, and apply an exponential at the end. E.g., the log of the homophily potential (slide 21) is w in the case A_i=A_j, and 0 otherwise. Also, all real numbers are then permissible return values, without the non-negativity condition.

The following are examples of a simple node (log-)potential and an "ising-style" homophily potential. Both potential functions depend on numerical parameters w1,w2. You may start with using these two log-potentials only, and later define additional potential functions.

In [ ]:
def n_log_potential_1(n,w1,w2):
    if n['gender'] == 1: return w1
    else: return w2

In [ ]:
def n_log_potential_2(n1,n2,w1,w2):
    if n1['predicted_practice']==n2['predicted_practice']: return w1
    else: return w2

In [ ]:
print(n_log_potential_1(lazega.nodes[5],-1,1))
print(n_log_potential_2(lazega.nodes[5],lazega.nodes[6],-2,3))

**Task 2:** Write a Gibbs-sampling function for re-sampling the 'predicted_practice' attribute value for a node *n*. At this point we need not worry whether for *n* the actual 'practice' value is known or not.

In [ ]:
def gibbs_sample(n):
    # Iterate over all log-potential functions you want to use
    # For node potentials, evaluate the potential for the given node n
    # For edge potentials, evaluate the potential for all pairs (n,n') where n' is a neighbor in the 'friendship' graph.
    #
    # Since the friendship graph is directed, there are three possibilities of how to do this precisely:
    #        - consider all n' where friends(n,n')
    #        - consider all n' where friends(n',n)
    #        - consider both cases of n'
    # 
    # The method nx.all_neighbors(lazega,n) will return an iterator over both types of neighbors of n, so the third
    # option is the most convenient to use (and maybe also the most sensible)
    #
    # Sum the values of all the potential functions, and take the exponential.
    # This has to be done for both possible values of the 'predicted_practice' value for n
    #
    # When the potential functions are defined with some free numerical parameters (as w1,w2 above), then at 
    # some point these parameters must be set to fixed values.
    # 
    # Calculate the probability for predicted_practice(n) according to the quotient shown on slide 22 (see also 
    # the example on slide 28)
    #
    # Set the new value of predicted_practice(n) randomly according to the probabilities you have just computed. 

Once the resampling of the predicted_practice attribute for a single node is in place, the rest is quite straightforward: 

**Task 3**: Write a function that performs one round of Gibbs sampling, i.e., re-samples the predicted values for all nodes for which the class attribute is unknown.

In [ ]:
def gibbs_one_round(G):
    # Iterate over all nodes n in G for which observed_practice == nan, and re-sample its predicted_practice value

Now we can put everything together to use our model to predict the 'practice' attribute. 

**Task 4**: write code for doing the following:

In [ ]:
# Initialize with init_att
#
# Perform a number of gibbs_one_round(lazega) sampling steps
#
# (Maybe after a certain number of burn-in iterations): keep count of how often the predicted_practice value of nodes 
# with observed_practice == nan is in the two states 1 or 2.
# 
# Predict the unobserved practice values as the more probably state in the Gibbs sample
#
# Compare your predicted values against the true values

Now that things are up and running, we can explore some of the properties of the model and the Gibbs sampling procedure: 

**Task 5**: 

<ul>
<li> Perform the Gibbs-prediction procedure several times and explore how stable the prediction results are. Apart from the final categorical prediction, you can also consider the actual frequencies of the two 'predicted_practice" states in your samples</li>
    <li> Try different settings of the parameter values of the potential functions. Can you find a relationship between the parameter values, and the stability of the Gibbs sampling? </li>
    <li> Also try different parameter settings in order to optimize prediction accuracy. In a real application, one would learn the parameter values by likelihood optimization based on the labeled training nodes. This is outside the scope of what we can do here, so let's do a simple grid search over parameter values instead (ugly!) and evaluate by checking accuracy using the known actual labels for the test nodes (this evaluation would usually be done on a set of validation nodes that is distinct from the final test nodes, but we need not worry about this distinction here).
    </ul>